In [31]:
import json


class Song:
    def __init__(self,key,scale,tonic,chords, **kwargs):
        # self.absolute_root = self.note_to_midi(key)
        self.key = key
        self.scale = scale
        self.tonic = tonic
        self.allowed_keys_list = self.generated_keys_list()
        print(self.allowed_keys_list)
        self.chords = chords
        self.generated_absolute_chords()
    def generated_keys_list(self):
        oct_start = 0
        oct_end = 10
        get_key_shift = self.get_key_shift(self.scale)
        absolute_root = self.note_to_midi(self.tonic, octave=oct_start)
        first_octave = [x + absolute_root for x in get_key_shift]
        key_list = [x + (12 * i) for i in range(oct_end) for x in first_octave]
        return key_list
    def generated_absolute_chords(self):
        if len(self.chords) == 0: return 
        for c in self.chords:
            absolute_root = self.note_to_midi(self.tonic) 
            root_abs_position = self.allowed_keys_list.index(absolute_root) + c.root - 1 
            c.absolute_chord_position = [self.allowed_keys_list[root_abs_position],self.allowed_keys_list[root_abs_position+2],self.allowed_keys_list[root_abs_position+4]]
    def get_key_shift(self, mode):
        ks = {
            "major" : [0, 2, 4, 5, 7, 9, 11],
            "minor" : [0, 2, 3, 5, 7, 8, 10],
            "dorian" : [0, 2, 3, 5, 7, 9, 10],
            "locrian" : [0, 1, 3, 5, 6, 8, 10],
            "mixolydian" : [0, 2, 4, 5, 7, 9, 10],
            "harmonicMinor" : [0, 2, 3, 5, 7, 8, 11],
            "lydian" : [0, 2, 4, 6, 7, 9, 11],
            "phrygian" : [0, 1, 3, 5, 7, 8, 10],
            "phrygianDominant" : [0, 1, 3, 5, 7, 8, 9]
        }
        return ks[mode]
    def note_to_midi(self, note, octave=5):
        notes = {
            'C': 0,
            'Db': 1,
            'C#': 1,
            'D': 2,
            'Eb': 3,
            'D#': 3,
            'E': 4,
            'F': 5,
            'Gb': 6,
            'F#': 6,
            'G': 7,
            'Ab': 8,
            'G#': 8,
            'A': 9,
            'Bb': 10,
            'A#': 10,
            'B': 11
        }
        return notes[note] + (octave * 12)
    def __str__(self):
        return f"key: {self.scale} \t {self.tonic}"
    def __repr__(self):
        return f"key: {self.scale} \t {self.tonic}"


class Chord:
    def __init__(self, root, beat, duration, type, inversion, applied, adds, omits, alterations, suspensions, substitutions, pedal, alternate, borrowed, isRest, recordingEndBeat=None):
        self.root = root
        self.beat = beat
        self.duration = duration
        self.type = type
        self.inversion = inversion
        self.applied = applied
        self.adds = adds
        self.omits = omits
        self.alterations = alterations
        self.suspensions = suspensions
        self.substitutions = substitutions
        self.pedal = pedal
        self.alternate = alternate
        self.borrowed = borrowed
        self.isRest = isRest
        self.recordingEndBeat = recordingEndBeat
        self.absolute_chord_position = []
    def __repr__(self):
        roman_numerals = {1: 'I', 2: 'II', 3: 'III', 4: 'IV', 5: 'V', 6: 'VI', 7: 'VII'}
        roman = roman_numerals.get(self.root, '?')
        return f"({roman} : {self.beat}-{self.beat + self.duration - 1})"
    def __str__(self):
        roman_numerals = {1: 'I', 2: 'II', 3: 'III', 4: 'IV', 5: 'V', 6: 'VI', 7: 'VII'}
        roman = roman_numerals.get(self.root, '?')
        return f"({roman} : {self.beat}-{self.beat + self.duration - 1})"

class Note:
    def __init__(self, sd, octave, beat, duration, isRest=None, **kwargs):
        self.sd = sd
        self.octave = octave
        self.beat = beat
        self.duration = duration
        self.isRest = isRest
    def __str__(self):
        return f"({self.sd}-{self.octave}: {self.beat}-{self.beat + self.duration - 0.5})"
    def __repr__(self):
        return f"({self.sd}-{self.octave}: {self.beat}-{self.beat + self.duration - 0.5})"



def load_music_data(json_file_path):
    try:
        with open(json_file_path, 'r', encoding='utf-8-sig') as file:
            data = json.load(file)
    except FileNotFoundError:
        print("Error: The file was not found.")
        return None, None
    except json.JSONDecodeError:
        print("Error: There was an issue decoding the JSON file.")
        return None, None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None, None

    chords = [Chord(**chord_data) for chord_data in data.get('chords', [])]
    notes = [Note(**note_data) for note_data in data.get('notes', [])]

    # Extract the first key (assuming there's only one)
    key = data.get('keys', [{}])[0]

    # Get the scale and tonic fields
    scale = key.get('scale')
    tonic = key.get('tonic')
    print(tonic, scale)
    # print(song)
    song = Song(f"{tonic}-{scale}",scale,tonic,chords_list)

    return chords, notes, song


json_file_path = '/Users/Juan.Huerta/github/music_data/data/'
scale_tonic_counts = {}

for filename in os.listdir(json_file_path):
    if filename.endswith('.json'):
        file_path = os.path.join(json_file_path, filename)
        print(file_path)
        chords_list, notes_list, song = load_music_data(file_path)
        if chords_list is not None and notes_list is not None:
            # scale = song.scale
            # tonic = song.tonic
            print(chords_list[0].root)
            print(song.chords[0].absolute_chord_position)
            
            print(chords_list[1].root)
            print(song.chords[1].absolute_chord_position)

            break
    break
            # if scale and tonic:
            #     pair = f"{tonic}-{scale}"
            #     scale_tonic_counts[pair] = scale_tonic_counts.get(pair, 0) + 1
        # if scale:
        #         pair = f"{scale}"
        #         scale_tonic_counts[scale] = scale_tonic_counts.get(scale, 0) + 1
        # if len(chords_list)>0:
        #     for c in chords_list:
        #         pair = f"{c}"
        #         scale_tonic_counts[c] = scale_tonic_counts.get(c, 0) + 1
        #         break

# print(scale_tonic_counts)


# # Example usage:
# json_file_path = '/Users/Juan.Huerta/github/music_data/bw.json'
# chords_list, notes_list, song = load_music_data(json_file_path)
# if chords_list is not None and notes_list is not None:
#     print("Chords and notes loaded successfully.")
# else:
#     print("Failed to load chords or notes.")


# print(chords_list)

/Users/Juan.Huerta/github/music_data/data/here comes the sun-2.json
E minor
[4, 6, 7, 9, 11, 12, 14, 16, 18, 19, 21, 23, 24, 26, 28, 30, 31, 33, 35, 36, 38, 40, 42, 43, 45, 47, 48, 50, 52, 54, 55, 57, 59, 60, 62, 64, 66, 67, 69, 71, 72, 74, 76, 78, 79, 81, 83, 84, 86, 88, 90, 91, 93, 95, 96, 98, 100, 102, 103, 105, 107, 108, 110, 112, 114, 115, 117, 119, 120, 122]
1
[64, 67, 71]
5
[71, 74, 78]


In [29]:
class KeyShifts:
    major = [2, 2, 1, 2, 2, 2, 1]
    minor = [2, 1, 2, 2, 1, 2, 2]
    dorian = [2, 1, 2, 2, 2, 1, 2]
    locrian = [1, 2, 2, 1, 2, 2, 2]
    mixolydian = [2, 2, 1, 2, 2, 1, 2]
    harmonicMinor = [2, 1, 2, 2, 1, 3, 1]
    lydian = [2, 2, 2, 1, 2, 2, 1]
    phrygian = [1, 2, 2, 2, 1, 2, 2]
    phrygianDominant = [1, 2, 2, 2, 1, 2, 1]
    


In [ ]:
    def get_key_shift(self, mode):
        ks = {
            "major" : [2, 2, 1, 2, 2, 2, 1],
            "minor" : [2, 1, 2, 2, 1, 2, 2],
            "dorian" : [2, 1, 2, 2, 2, 1, 2],
            "locrian" : [1, 2, 2, 1, 2, 2, 2],
            "mixolydian" : [2, 2, 1, 2, 2, 1, 2],
            "harmonicMinor" : [2, 1, 2, 2, 1, 3, 1],
            "lydian" : [2, 2, 2, 1, 2, 2, 1],
            "phrygian" : [1, 2, 2, 2, 1, 2, 2],
            "phrygianDominant" : [1, 2, 2, 2, 1, 2, 1]
        }
        return ks[mode]
        